In [131]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings, StackedEmbeddings, BytePairEmbeddings
from flair.data import Sentence
import torch
from collections import Counter

import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

import re
import numpy as np

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

/Users/gw00207/miniconda3/envs/bk_scraper/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gw00207/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [132]:
word_embedding = WordEmbeddings('en')
stacked_embedding = StackedEmbeddings([WordEmbeddings('en'),BytePairEmbeddings('en')])
embedding = DocumentPoolEmbeddings([word_embedding])

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs100000.model


100%|██████████| 1987533/1987533 [00:00<00:00, 3763012.21B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs100000.d50.w2v.bin.tar.gz


100%|██████████| 19357958/19357958 [00:04<00:00, 4351344.77B/s]


In [ ]:
df['genre'].unique()

In [96]:
df = pd.read_csv('data/all.csv')
jungle = df[df['genre']=='jungle / footwork']

## words

In [202]:
all_word = jungle['description'].str.replace('\xa0|/', ' ', regex=True).str.cat(sep=' ')

In [206]:
clean = re.sub(r'\.|,|\n|‘|’|!|\]|\[|\)|\(|`|;|”|', '', all_word)
shortlist = [x[0] for x in Counter(clean.split(' ')).most_common(20000)]

In [208]:
tokens_without_sw = [word.lower() for word in shortlist if not word.lower() in stopwords.words() and word!='']

In [209]:
embeddings_list = []
meta = []
for i in tokens_without_sw:
    sentence = Sentence(i)
    word_embedding.embed(sentence)
    embed = sentence[0].embedding.numpy()
    if np.any(embed):
        embeddings_list.append(embed)
        meta.append(i)

In [210]:
len(meta)

2600

In [211]:
len(embeddings_list)

2600

In [212]:
pd.DataFrame(embeddings_list)[:-1].to_csv('word_tensors.tsv', sep='\t', index=False, header=False)

In [213]:
pd.Series(meta, name='word')[:-1].to_csv('word_meta.tsv', sep='\t', index=False, header=False)

## descriptions

In [61]:
df.dropna(subset=['description'], inplace=True)

In [71]:
def get_tensors(x):
    sentence = Sentence(x['description'])
    embedding.embed(sentence)
    return sentence.get_embedding()

In [72]:
df['embedding'] = df.progress_apply(get_tensors, axis=1)

In [81]:
df['np_embedding'] = df['embedding'].apply(lambda x: x.detach().numpy())

In [113]:
df['description'] = df['description'].str.replace('\n', ' ')

In [114]:
df[['artist','album','catalogue_no','genre','description']].to_csv('meta.tsv', sep='\t', index=False)
df['np_embedding'].apply(pd.Series).to_csv('tensors.tsv', sep='\t', index=False, header=False)